In [5]:
import pyspark
from importlib import reload

from pyspark.sql import SparkSession
import os
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')
from script.helper.db_conn import db_connection
from datetime import datetime
from datetime import timezone

from script.helper.conn_prop import connection_properties
from script.helper.init_spark import initiate_spark
import logging
import pandas as pd

def etl_log(log_msg: dict):
    """
    This function is used to save the log message to the database.
    """


    try:
        # create connection to database
        spark = initiate_spark()
        _,_,_,log_url = db_connection()
        _,_,_,cp_log=connection_properties()
        
        # convert dictionary to dataframe
        df_log = pd.DataFrame([log_msg])
        df_log=spark.createDataFrame(data=df_log)

        #extract data log
        df_log.write.jdbc(url=log_url,table="etl_log",mode="append",properties=cp_log)

    except Exception as e:
        print("Can't save your log message. Cause: ", str(e))


def log_success(step,process, source, table_name):
    log_msg = {
        "step" : step,
        "process" : process, 
        "status" : "success", 
        "source": source,
        "table_name": table_name,
        "etl_date": datetime.now(timezone.utc).astimezone()  # Current timestamp
        }
    etl_log(log_msg)


def log_error(step,process, source, table_name,error_msg):
    log_msg = {
        "step" : step,
        "process" : process, 
        "status" : "failed", 
        "source": source,
        "table_name": table_name,
        "error_msg": error_msg,
        "etl_date": datetime.now(timezone.utc).astimezone()  # Current timestamp
        }
    etl_log(log_msg)





In [6]:
log_error("aaa","aaa","ss","aaa", "ssssssssssssss")

In [4]:
log_success("aaa","aaa","ss","aaa")

In [3]:
# Inisialisasi SparkSession
spark = initiate_spark()

In [4]:
_,_,_,log_url = db_connection()


In [5]:
_,_,_,cp_log=connection_properties()


In [6]:
log_url


'jdbc:postgresql://target_db_container:5432/etl_log'

In [7]:
cp_log

{'user': 'postgres',
 'password': 'cobapassword',
 'driver': 'org.postgresql.Driver'}

In [8]:
table_name="bb"

In [34]:

log_msg = {
    "step" : "staging",
    "process" : "aaa", 
    "status" : "aaa", 
    "source": "failed",
    "table_name": table_name,
    "error_msg": "msg",
    "etl_date": datetime.now(timezone.utc).astimezone()  # Current timestamp
    
}
    


In [35]:
df_log = pd.DataFrame([log_msg])

In [36]:
df_log=spark.createDataFrame(data=df_log)

In [37]:
#df_log = pd.DataFrame([log_msg])

In [38]:
df_log.write.jdbc(url=log_url,table="etl_log",mode="append",properties=cp_log)

In [8]:
import pyspark
from importlib import reload

from pyspark.sql import SparkSession
import os
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')
from script.helper.db_conn import db_connection
from datetime import datetime
from datetime import timezone

from script.helper.conn_prop import connection_properties
from script.helper.init_spark import initiate_spark
import logging
import pandas as pd

In [9]:
# Inisialisasi SparkSession
spark = initiate_spark()

In [39]:

def extract_csv(filename,spark=spark):
    try: 
        print(f"===== Start Extracting {filename} data =====")
        global step
        global process
        source=filename.split('/')[-1]
        table_name=source.split('.')[0]
        
        df=spark.read.csv(filename,header=True)
        log_success(step,process,source,table_name)
        
        print(f"===== Success Extracting {filename} data =====")
        return df
    except Exception as e:
        log_error(step,process,source,table_name,str(e))
        print(f"====== Failed to Extract Data {filename} ======,\n {e}")





In [40]:
step="Data Warehouse"
process="Extract"

In [41]:

aa=extract_csv('../../data/peoplea.csv')

===== Start Extracting ../../data/peoplea.csv data =====
====== Failed to Extract Data ../../data/peoplea.csv ======,
 [PATH_NOT_FOUND] Path does not exist: file:/home/jovyan/work/data/peoplea.csv.


In [11]:
print(os.getcwd())

/home/jovyan/work/script/staging


In [47]:
def extract_database(table_name,spark=spark):
    try:
        print(f"===== Start Extracting {table_name} data =====")
        
        cp_src,_,_,_=connection_properties()
        src_url,_,_,_ = db_connection()
        df_metadata=spark.read.jdbc(src_url, table=table_name, properties=cp_src)
        log_success(step,process,table_name,table_name)
        print(f"===== Success Extracting {table_name} data =====")
        return df_metadata
    except Exception as e:
        print(f"====== Failed to Extract Data {table_name} ======,\n {e}")
        log_error(step,process,table_name,table_name,str(e))


In [48]:
bb=extract_database('company')

===== Start Extracting company data =====
===== Success Extracting company data =====


In [49]:
bb

DataFrame[office_id: int, object_id: string, description: string, region: string, address1: string, address2: string, city: string, zip_code: string, state_code: string, country_code: string, latitude: decimal(9,6), longitude: decimal(9,6), created_at: timestamp, updated_at: timestamp]

In [50]:
print(bb)

DataFrame[office_id: int, object_id: string, description: string, region: string, address1: string, address2: string, city: string, zip_code: string, state_code: string, country_code: string, latitude: decimal(9,6), longitude: decimal(9,6), created_at: timestamp, updated_at: timestamp]


In [51]:
bb.show()

+---------+---------+-----------------+--------------------+--------------------+----------+--------------+----------+----------+------------+---------+-----------+-------------------+-------------------+
|office_id|object_id|      description|              region|            address1|  address2|          city|  zip_code|state_code|country_code| latitude|  longitude|         created_at|         updated_at|
+---------+---------+-----------------+--------------------+--------------------+----------+--------------+----------+----------+------------+---------+-----------+-------------------+-------------------+
|        8|      c:8|                 |              SF Bay|959 Skyway Road, ...|          |    San Carlos|     94070|        CA|         USA|37.506885|-122.247573|2007-01-01 22:19:54|2007-01-01 22:19:54|
|        9|      c:9|     Headquarters|         Los Angeles|9229 W. Sunset Blvd.|          |West Hollywood|     90069|        CA|         USA|34.090368|-118.393064|2007-01-01 22:19